In [140]:
import pandas as pd
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from joblib import dump, load
import json
from sklearn.metrics import r2_score

# Pipelines

Para el funcionamiento del API se tienen dos pipelines principales, que incluyen un transformador personalizado y son cargados mediante joblib. Además, estos pipelines se exportaron del laboratorio anterior con el modelo entrenado.

In [212]:
pipe1 = load('preparacion.joblib')
pipe2 = load('prediccion.joblib')

El primer pipeline, contiene únicamente un transformador de preparación de los datos que permite eliminar los registros que se definieron como atípicos en el laboratorio anterior. Este transformador se encuentra a continuación:

In [213]:
pipe1

Pipeline(steps=[('preparacion', PreparacionTransformer())])

In [122]:
class PreparacionTransformer(BaseEstimator,TransformerMixin):
    def __init__(self):
        pass
    def fit(self, X, y=None):
        return self
    def transform(self, X, y=None):
        X_ = X.copy()
        X_ = X_[(X_['Life expectancy']>0) & (X_['BMI']>=15)
        & (X_['BMI']<=50) & (X_['Income composition of resources']>0)
        & (X_['Adult Mortality']>0) & (X_['thinness  10-19 years']<15)
        & (X_['thinness 5-9 years']<15) & (X_['HIV/AIDS']<1)
        & (X_['Adult Mortality']<400)]
        return X_

Por otro lado, el segundo pipeline se encarga del preparamiento de los datos para el modelo, es decir, seleccionar únicamente las columnas con las que fue entrenado, estandarizar los datos y finalmente, usar estas transformaciones para predecir mediante la regresión lineal entrenada.

In [214]:
pipe2

Pipeline(steps=[('initial',
                 ColumnTransformer(transformers=[('selector', 'passthrough',
                                                  ['Income composition of '
                                                   'resources',
                                                   'Schooling', 'BMI',
                                                   'Adult Mortality',
                                                   'HIV/AIDS',
                                                   'thinness  10-19 years',
                                                   'thinness 5-9 years'])])),
                ('scale', StandardScaler()), ('model', LinearRegression())])

Finalmente, se elaboraron dos funciones, cada una para resolver un requerimiento del API elaborado. La función "conjuntoDatos" permite evaluar varios registros para conocer el error R^2 del modelo para esa predicción, y la función "registro" permite predecir la expectativa de vida para un único registro.

In [182]:
def conjuntoDatos(X):
    try:
        register_t = pipe1.transform(X)
        X_ = register_t.drop('Life expectancy', axis = 1)
        y = register_t['Life expectancy']
        if (len(X_)==0):
            return "Después de realizar la limpieza de datos se identificó que todos los registros eran atípicos"
        prediction = pipe2.predict(X_)
    except KeyError:
        return "Las columnas de los datos no coinciden con las del modelo"
    except:
        return "Ocurrió un error evaluando los datos en el modelo"
    return round(pipe2.score(X_,y),3)

In [216]:
def registro(X):
    try:
        prediction = pipe2.predict(X)[0]
    except ValueError:
        return "No se encuentran todas las columnas necesarias para evaluar el registro con el modelo"
    except RuntimeError as e:
        return "Hay columnas que no corresponden con las que se entrenó el modelo"+e
    except:
        return "Ocurrió un error en la predicción"
    return prediction

In [ ]:
pd.read_json('json')

In [217]:
registro(pd.read_json('''
[
   { 
      "Adult Mortality":241.0, 
      "infant deaths":6, 
      "Alcohol":2.58, 
      "percentage expenditure":0.358651421, 
      "Hepatitis B":98.0, 
      "Measles":59, 
      "BMI":42.0, 
      "under-five deaths":8, 
      "Polio":98.0, 
      "Total expenditure":2.16, 
      "Diphtheria":98.0, 
      "HIV\\/AIDS":0.1, 
      "GDP":26.371428, 
      "Population":487137.0, 
      "thinness 10-19 years":3.3, 
      "thinness 5-9 years":3.3, 
      "Income composition of resources":0.0, 
      "Schooling":10.4 
   } 
] 
'''))

TypeError: can only concatenate str (not "RuntimeError") to str